# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

[]


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores': {'Hygeine': 20}}


# Use count_documents to display the number of documents in the result
print("Doc Count is: ", establishments.count_documents(query))

# Display the first document in the results using pprint
results = establishments.find(query) 
pprint(results[0])


Doc Count is:  0


IndexError: no such item for Cursor instance

In [ ]:
# Convert the result to a Pandas DataFrame
establishments_df = pd.dataframe(results)
# Display the number of rows in the DataFrame
rows_count = len(establishments_df.index)
# Display the first 10 rows of the DataFrame
establishments_df.head(10)

AttributeError: module 'pandas' has no attribute 'dataframe'

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_rating = {'LocalAuthorityName': {'$REGEX':'London'}, 'Rating Value': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
print("Doc Count is: ", establishments.count_documents(query_rating))

# Display the first document in the results using pprint
pprint(query_rating[0])

OperationFailure: $in needs an array, full error: {'ok': 0.0, 'errmsg': '$in needs an array', 'code': 2, 'codeName': 'BadValue'}

In [ ]:
# Convert the result to a Pandas DataFrame
establishments_rating_df = pd.dataframe(query_rating)

# Display the number of rows in the DataFrame
rows_count = len(establishments_rating_df.index)

# Display the first 10 rows of the DataFrame
establishments_rating_df.head(10)


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = establishments.find({'BusinessName': 'Penang Flavours'}), {'geocode': 'latitude'}
longitude = establishments.find({'BusinessName': 'Penang Flavours'}), {'geocode': 'longitude'}

query = {'RatingValue': '5', 'geocode.latitude': {'$gte': 'latitude' - degree_search, '$lte': 'latitude' + degree_search}, 
         'geocode.longitude': {'$gte': 'longitude' - degree_search, 'lte': 'longitude' + degree_search}
         
sort = sort_values = {'$sort': { 'geocode.latitude': -1, 'geocode.longitude': -1 }}

# Print the results
results = establishments.find(query).sort(sort)
pprint(results[0])

In [ ]:
# Convert result to Pandas DataFrame
degree_search_df = pd.dataframe(results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'BusinessName': {'Hygeine': "0"}}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {_id: 'LocalAuthorityName': '$classification'}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': -1}}

# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
print("Doc Count is: ", len(results))

# Print the first 10 results
pprint(results.head(10))

In [ ]:
# Convert the result to a Pandas DataFrame
pipeline_df = pd.dataframe(pipeline)

# Display the number of rows in the DataFrame
row_count = len(pipeline_df.index)

# Display the first 10 rows of the DataFrame
pipeline_df.head(10)